# Benchmark: raster sampling
A bunch of different ways of sampling data from rasters at points.

There's benchmarking code at the bottom.

In [ ]:
import warnings
import time
from math import floor

import geopandas as gpd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from affine import Affine
from rasterstats import zonal_stats
from rasterio.io import MemoryFile
import joblib
from numba import jit, njit, prange

# rasterstats

In [ ]:
def rasterstats(locs, cube, aff):
    dfs = []
    for d in range(cube.shape[0]):
        res = {}
        for v in range(cube.shape[1]):
            s = zonal_stats(locs, cube[d, v, :, :], stats="mean", affine=aff)
            res[v] = [x["mean"] for x in s]
        dfs.append(pd.DataFrame(res))
        break
    return pd.concat(dfs)

# Manual sampling

In [ ]:
def manual(locs, cube, aff):
    with MemoryFile() as m:
        h, w, d = cube.shape[-2], cube.shape[-1], cube.dtype
        ds = m.open(driver="GTiff", count=1, height=h, width=w, dtype=d, transform=aff,)
    dfs = []
    for idx, row in locs.iterrows():
        row, col = ds.index(row.geometry.x, row.geometry.y)
        res = cube[:, :, row, col]
        res = pd.DataFrame(res)
        dfs.append(res)
    return pd.concat(dfs)

# Latlon

In [ ]:
def latlon(locs, cube, aff):
    with MemoryFile() as m:
        h, w, d = cube.shape[-2], cube.shape[-1], cube.dtype
        ds = m.open(driver="GTiff", count=1, height=h, width=w, dtype=d, transform=aff,)
    dfs = []
    for i in range(len(locs)):
        row, col = ds.index(locs[i][0], locs[i][1])
        res = cube[:, :, row, col]
        res = pd.DataFrame(res)
        dfs.append(res)
    df = pd.concat(dfs)
    return df

# No pandas

In [ ]:
def nopandas(locs, cube, aff):
    xs, ys = locs
    with MemoryFile() as m:
        h, w, d = cube.shape[-2], cube.shape[-1], cube.dtype
        ds = m.open(driver="GTiff", count=1, height=h, width=w, dtype=d, transform=aff,)
    res = []
    for i in range(len(xs)):
        row, col = ds.index(xs[i], ys[i])
        res.append(cube[:, :, row, col])
    df = pd.DataFrame(np.concatenate(res))
    return df

# No rasterio

In [ ]:
def norasterio(locs, cube, aff):
    xs, ys = locs
    res = []
    for i in range(len(xs)):
        col, row = ~aff * (xs[i], ys[i])
        res.append(cube[:, :, floor(row), floor(col)])
    df = pd.DataFrame(np.concatenate(res))
    return df

# No affine

In [ ]:
def noaffine(locs, cube, aff):
    xs, ys = locs
    sa, sb, sc, sd, se, sf, _, _, _ = tuple(~aff)
    res = []
    for i in range(len(xs)):
        col, row = (xs[i] * sa + ys[i] * sb + sc, xs[i] * sd + ys[i] * se + sf)
        res.append(cube[:, :, floor(row), floor(col)])
    df = pd.DataFrame(np.concatenate(res))
    return df

# JiT

In [ ]:
@jit
def jitted(locs, cube, aff):
    xs, ys = locs
    sa, sb, sc, sd, se, sf, _, _, _ = tuple(~aff)
    res = []
    for i in range(len(xs)):
        col, row = (xs[i] * sa + ys[i] * sb + sc, xs[i] * sd + ys[i] * se + sf)
        res.append(cube[:, :, floor(row), floor(col)])
    df = pd.DataFrame(np.concatenate(res))
    return df

# njit

In [ ]:
@njit
def _njitted(xs, ys, cube, invaff):
    sa, sb, sc, sd, se, sf, _, _, _ = invaff
    num_points = len(xs)
    num_scenes = cube.shape[0]
    num_bands = cube.shape[1]
    avals = np.empty((num_points * num_scenes, num_bands), dtype=np.float32)
    for i in range(num_points):
        col, row = (xs[i] * sa + ys[i] * sb + sc, xs[i] * sd + ys[i] * se + sf)
        res = cube[:, :, floor(row), floor(col)]
        avals[i * num_scenes : (i + 1) * num_scenes, :] = res
    return avals


def njitted(locs, cube, aff):
    xs, ys = locs
    invaff = tuple(~aff)
    avals = _njitted(xs, ys, cube, invaff)
    df = pd.DataFrame(data=avals)
    return df

# parallel

In [ ]:
@njit(parallel=True)
def _parallel(xs, ys, cube, invaff):
    sa, sb, sc, sd, se, sf, _, _, _ = invaff
    num_points = len(xs)
    num_scenes = cube.shape[0]
    num_bands = cube.shape[1]
    avals = np.empty((num_points * num_scenes, num_bands), dtype=np.float32)
    for i in prange(num_points):
        col, row = (xs[i] * sa + ys[i] * sb + sc, xs[i] * sd + ys[i] * se + sf)
        res = cube[:, :, floor(row), floor(col)]
        avals[i * num_scenes : (i + 1) * num_scenes, :] = res
    return avals


def parallel(locs, cube, aff):
    xs, ys = locs
    invaff = tuple(~aff)
    avals = _parallel(xs, ys, cube, invaff)
    df = pd.DataFrame(data=avals)
    return df

# Benchmarking

In [ ]:
class Benchmarker:
    def __init__(self, nums, loops, locs, cube, aff):
        self.nums = nums
        self.loops = loops
        self.locs = locs
        self.cube = cube
        self.aff = aff
        self.times = pd.DataFrame(index=nums)
        self.results = pd.DataFrame(index=[0, 1, 2])

    def bench(self, func, prep=None):
        res = []
        print(func.__name__, end=": ")
        for num in nums:
            print(num, end="  ")
            best = 9e9
            for _ in range(self.loops):
                p = self.locs[:num]
                if prep:
                    p = prep(p)
                start = time.time()
                df = func(p, self.cube, self.aff)
                self.results[func.__name__] = df.head(1).T
                elapsed = time.time() - start
                if elapsed < best:
                    best = elapsed
            res.append(best)
        self.times[func.__name__] = res
        print()

In [ ]:
locs = gpd.read_file("data/locs.gpkg")
cube = joblib.load("data/cube.joblib")
aff = Affine.from_gdal(111.0, 0.5, 0.0, -7.5, 0.0, -0.5)

In [ ]:
def prep_xys(locs):
    return [(row.geometry.x, row.geometry.y) for _, row in locs.iterrows()]


def prep_xs(locs):
    xs = np.array([row.geometry.x for _, row in locs.iterrows()])
    ys = np.array([row.geometry.y for _, row in locs.iterrows()])
    return xs, ys

In [ ]:
nums = [1, 10, 100, 1000, 10000]
loops = 4
b = Benchmarker(nums, loops, locs, cube, aff)

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    b.bench(rasterstats)
    b.bench(manual)
    b.bench(latlon, prep_xys)
    b.bench(nopandas, prep_xs)
    b.bench(norasterio, prep_xs)
    b.bench(noaffine, prep_xs)
    b.bench(jitted, prep_xs)
    b.bench(njitted, prep_xs)
    b.bench(parallel, prep_xs)

In [ ]:
# rasterstats only did one day out of the year
b.times.rasterstats = b.times.rasterstats * 365

In [ ]:
b.times.to_csv("sample_times.csv")

In [ ]:
fix, ax = plt.subplots(figsize=(20, 10))
b.times.plot(ax=ax)
ax.set_ylim([0, 0.1])
plt.show()

In [ ]:
cols = b.results.columns
for c in cols:
    assert (b.results[c] == b.results[cols[0]]).all()